In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.\\data\\kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

.\data\kaggle\books.csv
.\data\kaggle\book_tags.csv
.\data\kaggle\BX-Book-Ratings.csv
.\data\kaggle\BX-Books.csv
.\data\kaggle\BX-Users.csv
.\data\kaggle\IMDB Dataset.csv
.\data\kaggle\IMDb movies.csv
.\data\kaggle\IMDb ratings.csv
.\data\kaggle\netflix_titles.csv
.\data\kaggle\ratings.csv
.\data\kaggle\star.png
.\data\kaggle\tags.csv
.\data\kaggle\to_read.csv
.\data\kaggle\upvote.png


In [12]:
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor,KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline,SVD,BaselineOnly,SVDpp,NMF,SlopeOne,CoClustering
from surprise.accuracy import rmse
from surprise import accuracy

In [13]:
users = pd.read_csv('.\\data\\kaggle\\BX-Users.csv', sep='\";\"', names=['User-ID', 'Location', 'Age'], encoding='latin-1', skiprows=1)
books = pd.read_csv('.\\data\\kaggle\\BX-Books.csv', sep='\";\"', names=['ISBN', 'Book-Title' ,'Book-Author','Year-Of-Publication', 'Publisher', 'Image-Url-S', 'Image-Url-M', 'Image-Url-L'], encoding='latin-1', skiprows=1)
ratings = pd.read_csv('.\\data\\kaggle\\BX-Book-Ratings.csv', sep='\";\"', names=['User-ID', 'ISBN', 'Book-Rating'], encoding='latin-1', skiprows=1)

C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [14]:
users['User-ID'] = users['User-ID'].str.replace("\"","")
users['Location'] = users['Location'].str.replace("\";NULL","")
users['Age'] = users['Age'].fillna("0")
users['Age'] = users['Age'].str.replace("\"","")
books['ISBN'] = books['ISBN'].str.replace("\"","")
books['Book-Title'] = books['Book-Title'].str.replace("\"","")
ratings['User-ID'] = ratings['User-ID'].str.replace("\"","")
ratings['Book-Rating'] = ratings['Book-Rating'].str.replace("\"","").astype(int)

In [15]:
quality_ratings = ratings[ratings['Book-Rating']!=0]
quality_book = quality_ratings['ISBN'].value_counts().rename_axis('ISBN').reset_index(name = 'Count')
quality_book = quality_book[quality_book['Count']>5]['ISBN'].to_list()
quality_ratings = quality_ratings[quality_ratings['ISBN'].isin(quality_book)]
quality_ratings

,User-ID,ISBN,Book-Rating
8,276744,038550120X,7
16,276747,0060517794,9
19,276747,0671537458,9
20,276747,0679776818,8
28,276754,0684867621,8
...,...,...,...
1149743,276688,0836218655,10
1149744,276688,0836236688,10
1149761,276704,0345386108,6
1149771,276704,0743211383,7


In [16]:
quality_user = quality_ratings['User-ID'].value_counts().rename_axis('User-ID').reset_index(name = 'Count')
quality_user = quality_user[quality_user['Count']>5]['User-ID'].to_list()
quality_ratings = quality_ratings[quality_ratings['User-ID'].isin(quality_user)]
quality_ratings

,User-ID,ISBN,Book-Rating
174,276847,3257200552,5
177,276847,3404148576,8
179,276847,3423071516,10
182,276847,3426029553,8
186,276847,3442413508,10
...,...,...,...
1149715,276688,0553575104,6
1149738,276688,0688156134,8
1149739,276688,0743202694,10
1149743,276688,0836218655,10


In [17]:
mean_rating_user = quality_ratings.groupby('User-ID')['Book-Rating'].mean().reset_index(name='Mean-Rating-User')
mean_data = pd.merge(quality_ratings, mean_rating_user, on='User-ID')
mean_data['Diff'] = mean_data['Book-Rating'] - mean_data['Mean-Rating-User']
mean_data['Square'] = (mean_data['Diff'])**2
norm_data = mean_data.groupby('User-ID')['Square'].sum().reset_index(name='Mean-Square')
norm_data['Root-Mean-Square'] = np.sqrt(norm_data['Mean-Square'])
mean_data = pd.merge(norm_data, mean_data, on='User-ID')
mean_data['Norm-Rating'] = mean_data['Diff']/(mean_data['Root-Mean-Square'])  
mean_data['Norm-Rating'] = mean_data['Norm-Rating'].fillna(0)
max_rating = mean_data.sort_values('Norm-Rating')['Norm-Rating'].to_list()[-1]
min_rating = mean_data.sort_values('Norm-Rating')['Norm-Rating'].to_list()[0]
mean_data['Norm-Rating'] = 5*(mean_data['Norm-Rating'] - min_rating)/(max_rating-min_rating)
mean_data['Norm-Rating'] = np.ceil(mean_data['Norm-Rating']).astype(int)
norm_ratings = mean_data[['User-ID','ISBN','Norm-Rating']]
mean_data.sort_values('Norm-Rating')

,User-ID,Mean-Square,Root-Mean-Square,ISBN,Book-Rating,Mean-Rating-User,Diff,Square,Norm-Rating
14735,124989,0.956522,0.978019,0061092096,9,9.956522,-0.956522,0.914934,0
71043,247429,0.956522,0.978019,0842314296,9,9.956522,-0.956522,0.914934,0
54488,213760,56.769231,7.534536,0671693816,3,7.692308,-4.692308,22.017751,1
20494,136011,27.714286,5.264436,055328942X,3,7.428571,-4.428571,19.612245,1
47555,198114,53.875000,7.339959,037541200X,1,6.625000,-5.625000,31.640625,1
...,...,...,...,...,...,...,...,...,...
98100,5815,21.333333,4.618802,0316693251,10,7.333333,2.666667,7.111111,5
98082,58067,12.000000,3.464102,0062512668,10,8.000000,2.000000,4.000000,5
76394,259430,29.666667,5.446712,0679781587,10,5.833333,4.166667,17.361111,5
76499,259734,15.428571,3.927922,0441003257,10,7.714286,2.285714,5.224490,5


In [18]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(norm_ratings[['User-ID', 'ISBN', 'Norm-Rating']], reader)

In [19]:
benchmark = []
for algorithm in [SVD(), 
                  SVDpp(), 
                  SlopeOne(), 
                  NMF(), 
                  NormalPredictor(), 
                  KNNBaseline(), 
                  KNNBasic(), 
                  KNNWithMeans(),
                  KNNWithZScore(), 
                  BaselineOnly(),
                  CoClustering()]:
    
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [20]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.623352,0.085228,0.117079
SVD,0.637421,2.520638,0.193955
SVDpp,0.641839,44.031848,1.677377
KNNBasic,0.650660,0.476374,0.591660
KNNBaseline,0.656883,0.549010,0.694889
KNNWithMeans,0.715815,0.535670,0.633391
KNNWithZScore,0.728849,0.628932,0.654878
SlopeOne,0.743806,1.546676,0.754634
NMF,0.750847,3.155848,0.160830


In [21]:
# Baseline

train_set, test_set = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options={'method': 'sgd','learning_rate': .00005, 'n_epochs':30, 'reg':0.5})
fit = algo.fit(train_set)
pred = fit.test(test_set)
accuracy.rmse(pred)

Estimating biases using sgd...
RMSE: 0.6213


0.6212710930844728

In [22]:
# SVD 

algo = SVD(reg_bi = 0.5, lr_bi=0.005)
fit = algo.fit(train_set)
pred = fit.test(test_set)
accuracy.rmse(pred)

RMSE: 0.6392


0.6391983992315424

In [23]:
recommend = algo.trainset
users_norm = list(set(norm_ratings['User-ID'].to_list()))
books_norm = list(set(norm_ratings['ISBN'].to_list()))
norm_ratings['User-ID'].unique()

array(['100009', '100053', '100066', ..., '99738', '99955', '99996'],
      dtype=object)

In [24]:
pred_users = [user for user in users_norm if recommend.knows_user(recommend.to_inner_uid(user))]
pred_books = []
for book in books_norm:
    try:
        if recommend.knows_item(recommend.to_inner_iid(book)):
            pred_books.append(book)
    except:
        pass


In [25]:
pred_users[:5]

['248656', '134278', '173291', '228579', '178032']

In [26]:
def recommend_books(user_id, count):
    result=[]
    for b in pred_books:
        result.append([b,algo.predict(user_id,b,r_ui=4).est])
    recom = pd.DataFrame(result, columns=['ISBN','Rating'])
    merge = pd.merge(recom,books, on='ISBN' )
    return merge.sort_values('Rating', ascending=False).head(count)

In [27]:
recommendation = recommend_books('36938', 5)

In [28]:
scoring = recommendation.sort_values('Year-Of-Publication')
view = "".join(["<span><img src='"+a+"'></span>" for a in scoring['Image-Url-M'].to_list()])
scoring[['Book-Title']]

,Book-Title
10111,To Kill a Mockingbird
3755,"Tales of the City (Tales of the City Series, V..."
5910,The Man from St. Petersburg (#1 Author's Class...
8981,Das Parfum: Die Geschichte Eines Morders
1333,Hot Springs (Earl Swagger Novels (Paperback))


In [29]:
view

"<span><img src='http://images.amazon.com/images/P/0446310492.01.MZZZZZZZ.jpg'></span><span><img src='http://images.amazon.com/images/P/0060964049.01.MZZZZZZZ.jpg'></span><span><img src='http://images.amazon.com/images/P/0451163516.01.MZZZZZZZ.jpg'></span><span><img src='http://images.amazon.com/images/P/3257228007.01.MZZZZZZZ.jpg'></span><span><img src='http://images.amazon.com/images/P/0671035452.01.MZZZZZZZ.jpg'></span>"